In [1]:
mkdir -p ~/aiffel/word_embedding

# 1. 원핫 인코딩으로 벡터화
- 모든 단어의 관계를 독립적으로 정의하는 원-핫 인코딩이라는 방식도 있습니다. 
- 원-핫 인코딩을 하기 위해서는 우선 갖고 있는 텍스트 데이터에서 단어들의 집합인 단어장(vocabulary) 을 만듭니다. 그 후 단어장에 있는 모든 단어에 대해서 1부터 V까지 고유한 정수를 부여합니다. 
- 이 정수는 단어장에 있는 각 단어의 일종의 인덱스 역할을 합니다. 
- 각 단어는 V차원의 벡터로 표현되는데, 해당 단어의 인덱스 위치만 1이고 나머지는 전부 0의 값을 가지는 벡터가 됩니다.

## 1) 패키지 설치

In [2]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import re
from konlpy.tag import Okt
from collections import Counter

In [4]:
text = "임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어."
text

'임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어.'

## 2) 전처리
- 정규 표현식을 사용하여 특수문자들을 제거하고자 합니다.
-  일반적으로 자음의 범위는 'ㄱ ~ ㅎ', 모음의 범위는 'ㅏ ~ ㅣ'와 같이 지정할 수 있습니다.
- 한글, 공백을 제외한 모든 문자를 표현하는 regex : [^ㄱ-ㅎㅏ-ㅣ가-힣 ]

In [5]:
#reg (한글이 아닌 것) 모두를 공백으로 교체한다

reg = re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]")
text = reg.sub('', text)
print(text)

임금님 귀는 당나귀 귀 임금님 귀는 당나귀 귀 실컷 소리치고 나니 속이 확 뚫려 살 것 같았어


## 3) 토큰화
-  KoNLPy에 내장된 Okt 형태소 분석기를 사용해서 토큰 단위를 나눠준다

In [6]:
okt=Okt()
tokens = okt.morphs(text)
print(tokens)

['임금님', '귀', '는', '당나귀', '귀', '임금님', '귀', '는', '당나귀', '귀', '실컷', '소리', '치고', '나니', '속이', '확', '뚫려', '살', '것', '같았어']


## 4) 단어장 만들기
- 파이썬의 Counter 서브클래스를 사용해서 단어의 빈도를 카운트

In [7]:
vocab = Counter(tokens)
print(vocab)

Counter({'귀': 4, '임금님': 2, '는': 2, '당나귀': 2, '실컷': 1, '소리': 1, '치고': 1, '나니': 1, '속이': 1, '확': 1, '뚫려': 1, '살': 1, '것': 1, '같았어': 1})


In [9]:
vocab['임금님']

2

In [10]:
vocab_size = 5
vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 5개의 단어만 저장
print(vocab)

[('귀', 4), ('임금님', 2), ('는', 2), ('당나귀', 2), ('실컷', 1)]


-  most_common()는 상위 빈도수를 가진 단어를 주어진 수만큼 리턴

In [11]:
word2idx={word[0] : index+1 for index, word in enumerate(vocab)}
print(word2idx)

{'귀': 1, '임금님': 2, '는': 3, '당나귀': 4, '실컷': 5}


- 높은 빈도수를 가진 단어일수록 낮은 정수 인덱스를 부여

## 5) 원-핫 벡터 만들기

In [13]:
def one_hot_encoding(word, word2index):
       one_hot_vector = [0]*(len(word2index))
       index = word2index[word]
       one_hot_vector[index-1] = 1
       return one_hot_vector

In [14]:
one_hot_encoding("임금님", word2idx)

[0, 1, 0, 0, 0]

### 케라스를 통한 원-핫 인코딩(one-hot encoding)

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
print("임포트 완료")

임포트 완료


In [16]:
text = [['강아지', '고양이', '강아지'],['애교', '고양이'], ['컴퓨터', '노트북']]
text

[['강아지', '고양이', '강아지'], ['애교', '고양이'], ['컴퓨터', '노트북']]

In [17]:
t = Tokenizer()
t.fit_on_texts(text)
print(t.word_index) # 각 단어에 대한 인코딩 결과 출력.

{'강아지': 1, '고양이': 2, '애교': 3, '컴퓨터': 4, '노트북': 5}


- 케라스 토크나이저를 사용하면 주어진 텍스트로부터 단어장을 만들고, 단어장의 각 단어에 고유한 정수를 맵핑

In [21]:
vocab_size = len(t.word_index) + 1

In [19]:
sub_text = ['강아지', '고양이', '강아지', '컴퓨터']
encoded = t.texts_to_sequences([sub_text])
print(encoded)

[[1, 2, 1, 4]]


In [20]:
one_hot = to_categorical(encoded, num_classes = vocab_size)
print(one_hot)

[[[0. 1. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0.]]]


# 2. Word2Vec

In [23]:
!pip install nltk
!pip install gensim

In [24]:
import nltk
nltk.download('abc')
nltk.download('punkt')

[nltk_data] Downloading package abc to /aiffel/nltk_data...
[nltk_data]   Unzipping corpora/abc.zip.
[nltk_data] Downloading package punkt to /aiffel/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [26]:
from nltk.corpus import abc
corpus = abc.sents()

In [27]:
print(corpus[:3])

[['PM', 'denies', 'knowledge', 'of', 'AWB', 'kickbacks', 'The', 'Prime', 'Minister', 'has', 'denied', 'he', 'knew', 'AWB', 'was', 'paying', 'kickbacks', 'to', 'Iraq', 'despite', 'writing', 'to', 'the', 'wheat', 'exporter', 'asking', 'to', 'be', 'kept', 'fully', 'informed', 'on', 'Iraq', 'wheat', 'sales', '.'], ['Letters', 'from', 'John', 'Howard', 'and', 'Deputy', 'Prime', 'Minister', 'Mark', 'Vaile', 'to', 'AWB', 'have', 'been', 'released', 'by', 'the', 'Cole', 'inquiry', 'into', 'the', 'oil', 'for', 'food', 'program', '.'], ['In', 'one', 'of', 'the', 'letters', 'Mr', 'Howard', 'asks', 'AWB', 'managing', 'director', 'Andrew', 'Lindberg', 'to', 'remain', 'in', 'close', 'contact', 'with', 'the', 'Government', 'on', 'Iraq', 'wheat', 'sales', '.']]


In [28]:
print('코퍼스의 크기 :',len(corpus))

코퍼스의 크기 : 29059


In [29]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = corpus, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)
print("모델 학습 완료!")

모델 학습 완료!


- 이 corpus를 가지고 Word2Vec 훈련시키기

In [30]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.9233802556991577), ('skull', 0.9107487797737122), ('Bang', 0.9055431485176086), ('asteroid', 0.9051916003227234), ('third', 0.9019238352775574), ('baby', 0.8993036150932312), ('dog', 0.8985596299171448), ('bought', 0.8974378108978271), ('rally', 0.8911734819412231), ('dinosaur', 0.8890715837478638)]


In [31]:
# 모델 저장하고 로드하는 방법
from gensim.models import KeyedVectors

model.wv.save_word2vec_format('~/aiffel/word_embedding/w2v') 
loaded_model = KeyedVectors.load_word2vec_format("~/aiffel/word_embedding/w2v")
print("모델  load 완료!")

모델  load 완료!


In [32]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.9233802556991577), ('skull', 0.9107487797737122), ('Bang', 0.9055431485176086), ('asteroid', 0.9051916003227234), ('third', 0.9019238352775574), ('baby', 0.8993036150932312), ('dog', 0.8985596299171448), ('bought', 0.8974378108978271), ('rally', 0.8911734819412231), ('dinosaur', 0.8890715837478638)]


# 3. FastText

In [33]:
from gensim.models import FastText
fasttext_model = FastText(corpus, window=5, min_count=5, workers=4, sg=1)
print("FastText 학습 완료!")

FastText 학습 완료!


In [34]:
fasttext_model.wv.most_similar('overacting')

[('extracting', 0.93874591588974),
 ('lifting', 0.9318686723709106),
 ('malting', 0.931620717048645),
 ('mounting', 0.9306465983390808),
 ('shooting', 0.9303444623947144),
 ('fluctuating', 0.929731547832489),
 ('melting', 0.9294134378433228),
 ('reflecting', 0.9282941222190857),
 ('resolving', 0.9276877045631409),
 ('attracting', 0.9265851974487305)]

In [35]:
fasttext_model.wv.most_similar('memoryy')

[('memory', 0.94884192943573),
 ('mechanisms', 0.8743845820426941),
 ('mechanism', 0.8734592199325562),
 ('musical', 0.8589513897895813),
 ('basic', 0.8587403893470764),
 ('technical', 0.8539524078369141),
 ('intelligence', 0.8514528870582581),
 ('imagine', 0.8460912108421326),
 ('mechanical', 0.8449276089668274),
 ('perform', 0.8426198363304138)]

- 단어장에 없지만 가장 유사한 단어 10개를 출력해준다

# 4. GloVe
- 글로브(Global Vectors for Word Representation, GloVe) 는 2014년에 미국 스탠포드 대학에서 개발한 워드 임베딩 방법론입니다. 
- 워드 임베딩의 두 가지 접근 방법인 카운트 기반과 예측 기반 두 가지 방법을 모두 사용했다 는 것이 특징입니다.
- 다만, GloVe는 2014년에 개발되었고, 2015년에 1.2 버전이 나온 이후로는 관리되지 않고 있기 때문에 최신 버전의 python에서는 GloVe를 설치하는 것이 불가능합니다.

In [36]:
import gensim.downloader as api
glove_model = api.load("glove-wiki-gigaword-50")  # glove vectors 다운로드
glove_model.most_similar("dog")  # 'dog'과 비슷한 단어 찾기

[('cat', 0.9218004941940308),
 ('dogs', 0.8513158559799194),
 ('horse', 0.7907583713531494),
 ('puppy', 0.7754920721054077),
 ('pet', 0.7724708318710327),
 ('rabbit', 0.7720814347267151),
 ('pig', 0.7490062117576599),
 ('snake', 0.7399188876152039),
 ('baby', 0.7395570278167725),
 ('bite', 0.7387937307357788)]

In [37]:
glove_model.most_similar('overacting')

[('impudence', 0.7842012047767639),
 ('puerile', 0.7816032767295837),
 ('winningly', 0.7644237875938416),
 ('grossness', 0.7576098442077637),
 ('deconstructions', 0.748936653137207),
 ('over-the-top', 0.7460805773735046),
 ('buffoonery', 0.746045708656311),
 ('impetuosity', 0.7415392398834229),
 ('sophomoric', 0.736961841583252),
 ('zaniness', 0.7353197336196899)]

In [38]:
glove_model.most_similar('memoryy')

KeyError: "Key 'memoryy' not present"

- GloVe는 Word2Vec과 같이 OOV 문제를 가지고 있어서 'memoryy'라는 단어는 인식하지 못합니다. 또한 pre-trained GloVe 모델은 한글이나 알파벳 대문자가 포함된 데이터셋으로 학습하지 않았기 때문에 알파벳 소문자만 인식한다는 사실에 유의하세요!